# Learning and Decision Making

## Laboratory 6: Reinforcement learning

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. The windy gridworld domain

Consider the larger version of the windy gridworld domain depicted in the figure below.

<img src="windy.png" width="400px">

In it, a boat must navigate a 7 &times; 10 gridworld, to reach the goal cell, marked with _G_. There is a crosswind upward through the middle of the grid, in the direction indicated by the gray arrows. The boat has available the standard four actions -- _Up_, _Down_, _Left_ and _Right_. In the region affected by the wind, however, the resulting next state is shifted upward as a consequence of the crosswind, the strength of which varies from column to column. The strength of the wind is given below each column, and corresponds to the number of cells that the movement is shifted upward. For example, if the boat is one cell to the right of the goal, then the action _Left_ takes you to the cell just above the goal.

The agent pays a cost of 1 in every step before reaching the goal. The problem can be described as an MDP $(\mathcal{X},\mathcal{A},\mathbf{P},c,\gamma)$ as follows.

In [1]:
%matplotlib notebook
import numpy as np
import numpy.linalg as la
import matplotlib.pyplot as plt

np.set_printoptions(threshold=10)

# Problem specific parameters
WIND = (0, 0, 0, 1, 1, 1, 2, 2, 1, 0)
nrows = 7
ncols = 10
init = [3, 0]
goal = [3, 7]

# States
X = [[x, y] for x in range(nrows) for y in range(ncols)]
nX = len(X)

# Actions
A = ['U', 'D', 'L', 'R']
nA = len(A)

# Transition probabilities
P = dict()
P['U'] = np.zeros((nX, nX))
P['D'] = np.zeros((nX, nX))
P['L'] = np.zeros((nX, nX))
P['R'] = np.zeros((nX, nX))

for i in range(len(X)):
    x = X[i]
    y = dict()
    
    
    y['U'] = [x[0] - WIND[x[1]] - 1, x[1]]
    y['D'] = [x[0] - WIND[x[1]] + 1, x[1]]
    y['L'] = [x[0] - WIND[x[1]], x[1] - 1]
    y['R'] = [x[0] - WIND[x[1]], x[1] + 1]
    
    for k in y:
        y[k][0] = max(min(y[k][0], nrows - 1), 0)
        y[k][1] = max(min(y[k][1], ncols - 1), 0)
        j = X.index(y[k])
        P[k][i, j] = 1

c = np.ones((nX, nA))
c[X.index(goal), :] = 0

gamma = 0.99

# -- Pretty print

print('\n- MDP problem specification: -\n')

print('States:')
print(np.array(X))

print('\nActions:')
print(A)

print('\nTransition probabilities:')
for a in A:
    print('Action', a)
    print(P[a])
    
print('\ncost:')
print(c)

print('\nStart state:', init)
print('\nGoal state:', goal)


- MDP problem specification: -

States:
[[0 0]
 [0 1]
 [0 2]
 ..., 
 [6 7]
 [6 8]
 [6 9]]

Actions:
['U', 'D', 'L', 'R']

Transition probabilities:
('Action', 'U')
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]
('Action', 'D')
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]
 [ 0.  0.  0. ...,  0.  0.  1.]]
('Action', 'L')
[[ 1.  0.  0. ...,  0.  0.  0.]
 [ 1.  0.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  1.  0.]]
('Action', 'R')
[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0.  0.  1. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  

---

#### Activity 1.        

Compute the optimal _Q_-function for the MDP defined above using value iteration. As your stopping condition, use an error between iterations smaller than `1e-8`.

---

In [2]:
# Insert your code here.

import numpy as np 

i = 0

Q_Optimal = np.zeros((70,4))
   
err = 1

# Q-Function computation

#while i < 10:
while err > 1e-8:

    Q_Up = c[:,0][np.newaxis, :].T + gamma * P["U"].dot(Q_Optimal)
    Q_Down = c[:,1][np.newaxis, :].T + gamma * P["D"].dot(Q_Optimal)
    Q_Left = c[:,2][np.newaxis, :].T + gamma * P["L"].dot(Q_Optimal)
    Q_Right = c[:,3][np.newaxis, :].T + gamma * P["R"].dot(Q_Optimal)
        
    Qnew = np.min((Q_Up, Q_Down, Q_Left, Q_Right), axis=0)

    err = np.linalg.norm(Qnew - Q_Optimal)
        

    Q_Optimal = Qnew
        

    i += 1
        

print "Iterations:", i
print Q_Optimal





Iterations: 2107
[[ 88.86732306  88.86732306  88.86732306  88.86732306]
 [ 88.75487178  88.75487178  88.75487178  88.75487178]
 [ 88.64128462  88.64128462  88.64128462  88.64128462]
 ..., 
 [ 87.18534608  87.18534608  87.18534608  87.18534608]
 [ 87.31349262  87.31349262  87.31349262  87.31349262]
 [ 87.44035769  87.44035769  87.44035769  87.44035769]]


---

#### Activity 2.        

Write down a Python function that, given a Q-function $Q$ and a state $x$, selects a random action using the $\epsilon$-greedy policy obtained from $Q$ for state $x$. Your function should receive an optional parameter, corresponding to $\epsilon$, with default value of 0.1. 

**Note:** In the case of two actions with the same value, your $\epsilon$-greedy policy should randomize between the two.

---

In [3]:
# Insert your code here.
import numpy as np
import random
#t = np.array([[1,1,1]])

def select_action(Q,nX,epsilon = None):
    if epsilon is None:
        epsilon = 0.1
    
    #print "e:", e_value
    
   
    r = random.random()
    
    state_index = X.index(nX)
    
    if r> epsilon:
        
        # primeiro verifica qual é o min da lista depois verifica se existem mais do que 1 min
        # nesse caso vai guardar os indices desses valores para depois escolher um aleatoriamente
    
        aux=[]
        for i,e in enumerate(Q[state_index]):
            if e == np.amin(Q[state_index]):
                aux += [i] 


        if len(aux)>1:
            action_index = random.choice(aux)
        else:
            action_index = aux[0]   

    else:
        action_index = random.choice([0,1,2,3])
        
 
    
    # pega no indice obtido e escolhe a acçao correspondente
    selected_action = A[action_index]
    
    return selected_action

    
select_action(Q_Optimal,[0,0])

'R'

### 2. Model-based learning

You will now run the model-based learning algorithm discussed in class, and evaluate its learning performance.

# ---

#### Activity 3.        

Run the model-based reinforcement learning algorithm discussed in class to compute $Q^*$ for 100,000 iterations. Initialize each transition probability matrix as the identity and the cost function as all-zeros. Use an $\epsilon$-greedy policy with $\epsilon=0.1$ (use the function from Activity 2). Note that, at each step,

* You will need to select an action according to the $\epsilon$-greedy policy;
* The state and action, you will then compute the cost and generate the next state; 
* With this transition information (state, action, cost, next-state), you can now perform an update. 
* When updating the components $(x,a)$ of the model, use the step-size

$$\alpha_t=\frac{1}{N_t(x,a)+1},$$

where $N_t(x,a)$ is the number of visits to the pair $(x,a)$ up to time step $t$.

In order to ensure that your algorithm visits every state and action a sufficient number of times, after the boat reaches the goal cell, make one further step, the corresponding update, and then reset the position of the boat to a random state in the environment.

Plot the norm $\|Q^*-Q^{(k)}\|$ every 500 iterations of your method, where $Q^*$ is the optimal _Q_~function computed in Activity 1.

**Note:** The simulation may take a bit. Don't despair.

---

In [21]:
# Insert your code here.

# Inicializar probabilidades de transiçao a 0
P_Up = np.eye(nX, nX)
P_Down = np.eye(nX, nX)
P_Left = np.eye(nX, nX)
P_Right = np.eye(nX, nX)

# Inicializar custos a 0
cost = np.zeros((nX, nA))

state_visits = np.zeros((70,4))


def calc_step_size(current_state, action):
    return 1 / ( state_visits[X.index(current_state), A.index(action)] + 1 )
    

def update_state(current_state, action):
    
# AS COORDENADAS DO PROF ESTAO COMO [Y,X]

    if action == "U":
        
        if WIND[current_state[1]]==0:
            new_state = [current_state[0] + 1, current_state[1]]
        elif WIND[current_state[1]]==1:
            new_state = [current_state[0] + 2, current_state[1]]
        else:
            new_state = [current_state[0] + 3, current_state[1]]
        
        if new_state[0]>6:
            new_state[0]=6
            
        
    elif action == "D":
        
        if WIND[current_state[1]]==0:
            new_state = [current_state[0] - 1, current_state[1]]
        elif WIND[current_state[1]]==1:    
            new_state = [current_state[0], current_state[1]]
        else:
            new_state = [current_state[0] + 1, current_state[1]]
        
        if new_state[0]>6:
            new_state[0]=6
        if new_state[0]<0:
            new_state[0]=0
            
    elif action == "L":
      
        if WIND[current_state[1]]==0:
            new_state = [current_state[0], current_state[1] - 1]
        elif WIND[current_state[1]]==1:
            new_state = [current_state[0] + 1, current_state[1] - 1]
        else:
            new_state = [current_state[0] + 2, current_state[1] - 1]
            
        if new_state[0]>6:
            new_state[0]=6
        if new_state[1]<0:
            new_state[1]=0
        
    elif action == "R":
        if WIND[current_state[1]]==0:
            new_state = [current_state[0], current_state[1] + 1]
        elif WIND[current_state[1]]==1:
            new_state = [current_state[0] + 1, current_state[1] + 1]
        else:
            new_state = [current_state[0] + 2, current_state[1] + 1]
            
        if new_state[0]>6:
            new_state[0]=6
        if new_state[1]>9:
            new_state[1]=9
    

    return new_state
    
    

##############################################################################
##############################################################################
##############################################################################

i = 0

Q_k = np.zeros((70,4))

goal_reached = False

current_state = init

norms_RL = []

while i < 100000: # demora uns 30 segundos a correr 100000 iteraçoes
    
    
    # Select Action
    action = select_action(Q_k,current_state)
    
    # Update Visit
    state_visits[X.index(current_state), A.index(action)] += 1
    
    # Calculate Step Size
    step_size = calc_step_size(current_state, action)
    
    # Update Cost
    cost[X.index(current_state), A.index(action)] = cost[X.index(current_state), A.index(action)] \
                                                    + step_size * (c[X.index(current_state), A.index(action)] \
                                                    - cost[X.index(current_state), A.index(action)])
    
    # Update Q
    Q_Up = cost[:,0][np.newaxis, :].T + gamma * P_Up.dot(Q_k)
    Q_Down = cost[:,1][np.newaxis, :].T + gamma * P_Down.dot(Q_k)
    Q_Left = cost[:,2][np.newaxis, :].T + gamma * P_Left.dot(Q_k)
    Q_Right = cost[:,3][np.newaxis, :].T + gamma * P_Right.dot(Q_k)
        
    Qnew = np.min((Q_Up, Q_Down, Q_Left, Q_Right), axis=0)
        
    Q_k = Qnew
    
    
    if i%500 == 0:
        n = np.linalg.norm(Q_Optimal - Q_k)
        norms_RL += [n]

        
    
    if goal_reached == True:
        
        #print "JA PASSAMOS PELO GOAL"
        
        next_state = random.choice(X)

        
        current_state = next_state
        
    else:
         

        next_state = update_state(current_state, action)

        current_state = next_state  
    
    if current_state == goal:
        goal_reached = True
        
        
    i += 1
        

print "Iterations:", i
print norms_RL
print "done"





Iterations: 100000
[1475.8140863399572, 1353.795917947976, 1352.0867231575228, 1251.7763265567962, 1181.5919127454956, 1145.5346137062381, 1117.0166579773677, 1088.2168228705989, 1085.9759732598779, 1067.9192401745961, 1051.1901398847772, 1034.7002751612492, 896.89786719888104, 575.40432918350189, 316.85450529434507, 241.39691009640262, 201.79476172740834, 198.23214062698463, 199.99050824221482, 202.76996033399158, 205.80805340071001, 209.34546615549093, 212.62176182931393, 215.52953263114676, 217.8356514587077, 220.62268862272234, 223.25040993282039, 225.45083825385302, 227.13175558291525, 228.8297647356365, 230.22989185031463, 231.56708238691766, 232.87275976437743, 234.02411244603985, 235.1499246295796, 236.16308963226382, 237.1304003279194, 237.95741342596813, 238.82392032925918, 239.5758767920409, 240.29099796757828, 241.04141190861301, 241.62471500220093, 242.20654299429586, 242.82852846307918, 243.34950643096013, 243.78957142728831, 244.27797710477134, 244.69294833942283, 245.13

### 3. Temporal-difference learning

You will now run both Q-learning and SARSA, and compare their learning performance with that of the model-based method just studied.

---

#### Activity 4.        

Repeat Activity 3 but using the _Q_-learning algorithm with a learning rate $\alpha=0.3$.

---

In [18]:
# Insert your code here.

# Inicializar probabilidades de transiçao a 0
P_Up = np.eye(nX, nX)
P_Down = np.eye(nX, nX)
P_Left = np.eye(nX, nX)
P_Right = np.eye(nX, nX)

# Inicializar custos a 0
cost = np.zeros((nX, nA))

state_visits = np.zeros((70,4))
        

##############################################################################
##############################################################################
##############################################################################

i = 0

Q_k = np.zeros((70,4))

goal_reached = False

current_state = init

norms_QL = []

while i < 100000: # demora uns 30 segundos a correr 100000 iteraçoes
    
    
    # Select Action
    action = select_action(Q_k,current_state)
    
    # Update Visit
    state_visits[X.index(current_state), A.index(action)] += 1
    
    # Calculate Step Size
    step_size = calc_step_size(current_state, action)
    
    # Update Cost
    cost[X.index(current_state), A.index(action)] = cost[X.index(current_state), A.index(action)] \
                                                    + step_size * (c[X.index(current_state), A.index(action)] \
                                                    - cost[X.index(current_state), A.index(action)])
    
    
    
    
    if i%500 == 0:
        n = np.linalg.norm(Q_Optimal - Q_k)
        norms_QL += [n]

        
    
    if goal_reached == True:
        
        #print "JA PASSAMOS PELO GOAL"
        
        next_state = random.choice(X)
        
        current_state = next_state
        
    else:
         

        next_state = update_state(current_state, action)

        current_state = next_state 
        
    
    # Update Q
    Q_k[X.index(current_state), A.index(action)] = Q_k[X.index(current_state), A.index(action)] + \
                                                   alpha * (cost[X.index(current_state), A.index(action)] + \
                                                   gamma * min(Q_k[X.index(next_state),:]) - \
                                                   Q_k[X.index(current_state), A.index(action)])
    
    if current_state == goal:
        goal_reached = True
        

    i += 1
        

print "Iterations:", i
print norms_QL
print "done"
    


Iterations: 100000
[1475.8140863399572, 1475.4192139311742, 1475.1198832001601, 1474.8197943256882, 1474.4556860302876, 1474.2095279825437, 1473.8386621156867, 1473.5578312570535, 1473.1638316829535, 1472.8206043431601, 1472.614827676186, 1472.4256838514527, 1472.1772071931728, 1471.851464331527, 1471.6039475520997, 1471.3451913071497, 1471.0326866081571, 1470.7473452978488, 1470.5042644499904, 1470.2215281435128, 1469.991711839984, 1469.7558050692212, 1469.5652664889094, 1469.3630636041964, 1469.1347969973981, 1468.8923074872466, 1468.7278191249984, 1468.5619045488058, 1468.3419265897473, 1468.1760249709052, 1467.9660403065222, 1467.7952141291876, 1467.6186587277446, 1467.4203330194189, 1467.2281758800284, 1467.0135574102858, 1466.7439148325291, 1466.4222492936567, 1466.1818648894057, 1465.9945927608765, 1465.8105621123573, 1465.6098903395682, 1465.4222073526216, 1465.3095224904082, 1465.1506518331923, 1465.0325776830412, 1464.8667316902779, 1464.7294346289691, 1464.5784092897925, 146

---

#### Activity 5.

Repeat Activity 4 but using the SARSA algorithm.

---

In [16]:
# Insert your code here.

# Inicializar probabilidades de transiçao a 0
P_Up = np.eye(nX, nX)
P_Down = np.eye(nX, nX)
P_Left = np.eye(nX, nX)
P_Right = np.eye(nX, nX)

# Inicializar custos a 0
cost = np.zeros((nX, nA))

state_visits = np.zeros((70,4))
        

##############################################################################
##############################################################################
##############################################################################

i = 0

Q_k = np.zeros((70,4))

goal_reached = False

current_state = init

norms_S = []

while i < 100000: # demora uns 30 segundos a correr 100000 iteraçoes
    
    
    # Select Action
    action = select_action(Q_k,current_state)
    
    # Update Visit
    state_visits[X.index(current_state), A.index(action)] += 1
    
    # Calculate Step Size
    step_size = calc_step_size(current_state, action)
    
    # Update Cost
    cost[X.index(current_state), A.index(action)] = cost[X.index(current_state), A.index(action)] \
                                                    + step_size * (c[X.index(current_state), A.index(action)] \
                                                    - cost[X.index(current_state), A.index(action)])
    
    
    
    
    if i%500 == 0:
        n = np.linalg.norm(Q_Optimal - Q_k)
        norms_S += [n]
        
    
    if goal_reached == True:
        
        #print "JA PASSAMOS PELO GOAL"
        
        next_state = random.choice(X)
        
        current_state = next_state
        
    else:
         

        next_state = update_state(current_state, action)

        current_state = next_state 
      
    next_action = select_action(Q_k, next_state)
    
    # Update Q
    Q_k[X.index(current_state), A.index(action)] = Q_k[X.index(current_state), A.index(action)] + \
                                                   alpha * (cost[X.index(current_state), A.index(action)] + \
                                                   gamma * Q_k[X.index(next_state), A.index(next_action)] - \
                                                   Q_k[X.index(current_state), A.index(action)])
    
    if current_state == goal:
        goal_reached = True
        

    i += 1
        

print "Iterations:", i
print norms_S
print "done"

Norm: 1475.81408634
Norm: 1475.08684843
Norm: 1474.70619788
Norm: 1474.31278105
Norm: 1473.94405026
Norm: 1473.72881907
Norm: 1473.24057335
Norm: 1473.00446206
Norm: 1472.83119857
Norm: 1472.67886702
Norm: 1472.45686034
Norm: 1472.26188881
Norm: 1472.08332377
Norm: 1471.93459204
Norm: 1471.86499718
Norm: 1471.71244356
Norm: 1471.55244243
Norm: 1471.46287428
Norm: 1471.37076561
Norm: 1471.22627839
Norm: 1471.17739521
Norm: 1471.00274996
Norm: 1470.88287642
Norm: 1470.7396494
Norm: 1470.52369346
Norm: 1470.38710664
Norm: 1470.27884792
Norm: 1470.10157696
Norm: 1469.88390745
Norm: 1469.7548984
Norm: 1469.56805585
Norm: 1469.35674469
Norm: 1469.17624234
Norm: 1469.100064
Norm: 1468.8482554
Norm: 1468.63373481
Norm: 1468.46792651
Norm: 1468.38404902
Norm: 1468.35362522
Norm: 1468.31175914
Norm: 1468.20035858
Norm: 1468.00597585
Norm: 1467.76433032
Norm: 1467.43371247
Norm: 1467.31617939
Norm: 1467.17154504
Norm: 1466.94958707
Norm: 1466.83711749
Norm: 1466.72995986
Norm: 1466.50090192
Norm:

---

#### Activity 6.

Discuss the differences observed between the performance of the three methods.

---

_Add your discussion here._